In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install beautifulsoup4

In [2]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
os.environ["TAVILY_API_KEY"] = "your-api-key"
os.environ["OPENAI_API_KEY"] = 'your-api-key'

# Building a Research and Report Agent

Steps:

1. Setup the LLM
2. Define the tool or list of tools
3. Set up a prompt template
4. Bind the llm with the tools
6. Use LCEL to connect the agent with prompt and the LLM binded with tools
7. Create the agent loop
8. Wrap everything under `AgentExecutor`
9. Invoke the agent with some query input

## 1. Setup the LLM

In [1]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None}, id='run-d77b6817-7d0d-45f4-9b4c-492f3e380f37-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

## 2. Define the tool or list of tools

In [2]:
import requests
from bs4 import BeautifulSoup
import os

# Tools: search and scraping an internet page
from langchain.tools import tool
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}


@tool
def search_tool(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    search = GoogleSearch({**serpapi_params, "q": query, "n": 5})
    output = search.get_dict()
    results = output["organic_results"]
    return [r["link"] for r in results]

@tool
def scrape_text(url: str):
    """Scrapes text from an internet page"""
    # source: copied from this script by Harrison Chase https://gist.github.com/hwchase17/69a8cdef9b01760c244324339ab64f0c
    try:
        response = requests.get(url)
        #check
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Extract all the text from the page
            page_text = soup.get_text(separator=" ", strip=True)
            
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code: {response.status_code}"
    
    except Exception as e:
        # In case of error print the error message
        print(e)
        return f"Failed to retrieve the webpage: {e}"

query = "Trends in using AI for education."
links = search_tool.invoke(query)
print(links)

url = links[0]

print(scrape_text.invoke(url))

['https://www.the74million.org/article/7-artificial-intelligence-trends-that-could-reshape-education-in-2024/', 'https://www.hurix.com/top-trends-reshaping-the-world-of-education-in/', 'https://www.forbes.com/sites/forbescommunicationscouncil/2024/01/23/how-ai-and-data-will-change-education-in-2024/', 'https://caktus.ai/blog/ai-in-education', 'https://www.weforum.org/agenda/2024/04/future-learning-ai-revolutionizing-education-4-0/', 'https://www2.ed.gov/documents/ai-report/ai-report.pdf', 'https://www.panopto.com/blog/five-ai-trends-that-are-good-news-for-higher-education/', 'https://www.sciencedirect.com/science/article/pii/S1877050918315382', 'https://news.stanford.edu/stories/2024/02/technology-in-education']
7 Artificial Intelligence Trends That Could Reshape Education in 2024 – The 74 The 74 America's Education News Source About The 74 Donate 7 Artificial Intelligence Trends That Could Reshape Education in 2024 News Opinion Home Video Analysis Search Donate Search Sections News Op

In [4]:
tools = [search_tool, scrape_text]

3. Set up a prompt template


In [5]:
from langchain.prompts import ChatPromptTemplate

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. \
                        Your sole purpose is to write well written, critically \
                        acclaimed, objective \
                        and structured reports on given text."
# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
Write a detailed research report on the following question:\n
"{input}". -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("placeholder"), "{chat_history}",
        ("human", RESEARCH_REPORT_TEMPLATE),
        ("placeholder"), "{agent_scratchpad}",
    ]
)

4. Bind the llm with the tools


In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers import ToolsAgentOutputParser

5. Use LCEL to define and connect agent, prompt and the LLM binded with tools


In [8]:
from langchain_core.runnables import RunnablePassthrough

agent = (
RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x['intermediate_steps'])
                    )
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

6. Create the agent loop

7. Wrap everything under `AgentExecutor`s

In [9]:
from langchain.agents import AgentExecutor

In [10]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

8. Invoke the agent with some query input

In [11]:
from IPython.display import Markdown

output_report = agent_executor.invoke({"input": "Write a quick report on the current trends in AI. Focus on LLMs for education.", "chat_history": []})

Markdown(output_report["output"])



> Entering new AgentExecutor chain...

Invoking: `search_tool` with `{'query': 'current trends in AI 2023'}`


['https://www.forbes.com/sites/ganeskesari/2022/12/22/ai-trends-for-2023-industry-experts-and-chatgpt-ai-make-their-predictions/', 'https://www.technologyreview.com/2023/12/19/1085696/four-trends-that-changed-ai-in-2023/', 'https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai-in-2023-generative-ais-breakout-year', 'https://www.bocasay.com/major-ai-trends-2023/', 'https://www.gartner.com/en/articles/what-s-new-in-artificial-intelligence-from-the-2023-gartner-hype-cycle', 'https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends', 'https://www.eweek.com/artificial-intelligence/ai-trends/', 'https://www.bakertilly.com/insights/ai-trends-2023-recap-and-insights-for-whats-to-come', 'https://appinventiv.com/blog/ai-trends/']
Invoking: `scrape_text` with `{'url': 'https://www.forbes.com/sites/ganeskesari/2022/12/22/ai-trends-

# Current Trends in AI: Focus on Large Language Models (LLMs) for Education

## Introduction

Artificial Intelligence (AI) has been a transformative force across various sectors, and its impact on education is becoming increasingly significant. Among the various AI technologies, Large Language Models (LLMs) like OpenAI's GPT-4 have shown immense potential in revolutionizing educational practices. This report delves into the current trends in AI with a specific focus on the application of LLMs in education, highlighting their benefits, challenges, and future prospects.

## Overview of AI Trends in 2023

### Generative AI

Generative AI, which includes LLMs, has seen explosive growth in 2023. These models are capable of generating human-like text, creating images, and even producing music. The widespread adoption of generative AI tools like ChatGPT, DALL-E, and others has brought AI into mainstream use, making it accessible to a broader audience (Kesari, 2022; Heikkilä, 2023).

### Multimodal AI

Multimodal AI, which processes multiple types of data inputs such as text, images, and audio, is another significant trend. This technology aims to mimic human sensory processing, enhancing the capabilities of AI systems to understand and interact with the world more comprehensively (Craig, 2024).

### AI Democratization

The democratization of AI refers to making AI tools accessible to a wider audience, including non-technical users. This trend is driven by the development of user-friendly AI platforms and open-source frameworks, enabling more people to leverage AI for various applications (Ayuya, 2024).

### AI in Cybersecurity

AI-driven cybersecurity solutions are becoming more prevalent, offering enhanced threat detection and response capabilities. These systems use machine learning to predict, detect, and respond to cyber threats more efficiently than traditional methods (Ayuya, 2024).

### Ethical and Explainable AI

As AI systems become more integrated into daily life, there is a growing emphasis on ethical AI and explainability. This involves developing AI systems that are transparent, fair, and accountable, addressing concerns about bias and misuse (Srivastava, 2024).

## LLMs in Education

### Enhancing Learning Experiences

LLMs have the potential to transform educational experiences by providing personalized learning, automating administrative tasks, and offering new ways to engage students. These models can generate customized content, answer student queries, and provide feedback, making learning more interactive and tailored to individual needs (McKinsey, 2023).

### Personalized Learning

One of the most significant benefits of LLMs in education is their ability to offer personalized learning experiences. By analyzing student data, LLMs can tailor educational content to meet the specific needs and learning styles of each student. This personalized approach can help improve student engagement and outcomes (Baker Tilly, 2024).

### Automating Administrative Tasks

LLMs can also automate various administrative tasks, such as grading assignments, scheduling, and managing student records. This automation can free up educators' time, allowing them to focus more on teaching and interacting with students (Appinventiv, 2024).

### Enhancing Accessibility

LLMs can make education more accessible by providing real-time translation services, generating content in multiple languages, and offering support for students with disabilities. For example, AI-powered tools can convert text to speech, provide subtitles for videos, and offer other assistive technologies (Bocasay, 2023).

## Challenges and Ethical Considerations

### Data Privacy and Security

The use of LLMs in education raises significant concerns about data privacy and security. Educational institutions must ensure that student data is protected and that AI systems comply with relevant regulations and standards (TechTarget, 2024).

### Bias and Fairness

LLMs can inadvertently perpetuate biases present in their training data, leading to unfair or discriminatory outcomes. It is crucial to develop and implement strategies to mitigate these biases and ensure that AI systems are fair and equitable (Appinventiv, 2024).

### Transparency and Explainability

Ensuring that AI systems are transparent and explainable is essential for building trust among educators, students, and parents. This involves making the decision-making processes of AI systems understandable and providing clear explanations for their outputs (Srivastava, 2024).

### Ethical Use of AI

The ethical use of AI in education involves considering the broader implications of AI technologies and ensuring that they are used responsibly. This includes addressing issues such as the potential for AI to replace human educators and the impact of AI on student well-being (Baker Tilly, 2024).

## Future Prospects

### Integration with Other Technologies

The future of LLMs in education will likely involve their integration with other emerging technologies, such as augmented reality (AR), virtual reality (VR), and the Internet of Things (IoT). These integrations can create more immersive and interactive learning experiences, further enhancing the educational process (Appinventiv, 2024).

### Lifelong Learning and Professional Development

LLMs can also play a significant role in lifelong learning and professional development. By providing personalized learning paths and resources, AI can help individuals continuously update their skills and knowledge, keeping pace with the rapidly changing job market (McKinsey, 2023).

### Research and Innovation

Ongoing research and innovation in AI and LLMs will continue to drive advancements in educational technologies. This includes developing more sophisticated models, improving the accuracy and reliability of AI systems, and exploring new applications of AI in education (Heikkilä, 2023).

## Conclusion

The application of LLMs in education represents a significant trend in the current AI landscape. These models have the potential to revolutionize educational practices by offering personalized learning experiences, automating administrative tasks, and enhancing accessibility. However, it is essential to address the challenges and ethical considerations associated with their use to ensure that AI technologies are implemented responsibly and effectively. As research and innovation continue to advance, the future of LLMs in education looks promising, with the potential to transform learning experiences and outcomes for students worldwide.

## References

- Kesari, G. (2022). AI Trends For 2023: Industry Experts (And ChatGPT AI) Make Their Predictions. Forbes. Retrieved from [Forbes](https://www.forbes.com/sites/ganeskesari/2022/12/22/ai-trends-for-2023-industry-experts-and-chatgpt-ai-make-their-predictions/)
- Heikkilä, M. (2023). Four trends that changed AI in 2023. MIT Technology Review. Retrieved from [MIT Technology Review](https://www.technologyreview.com/2023/12/19/1085696/four-trends-that-changed-ai-in-2023/)
- McKinsey & Company. (2023). The state of AI in 2023: Generative AI’s breakout year. Retrieved from [McKinsey](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai-in-2023-generative-ais-breakout-year)
- Bocasay. (2023). Major AI Trends in 2023. Retrieved from [Bocasay](https://www.bocasay.com/major-ai-trends-2023/)
- TechTarget. (2024). 10 top AI and machine learning trends for 2024. Retrieved from [TechTarget](https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends)
- Ayuya, C. (2024). 10 Most Impactful AI Trends in 2024. eWeek. Retrieved from [eWeek](https://www.eweek.com/artificial-intelligence/ai-trends/)
- Baker Tilly. (2024). AI trends: 2023 recap and insights for what’s to come. Retrieved from [Baker Tilly](https://www.bakertilly.com/insights/ai-trends-2023-recap-and-insights-for-whats-to-come)
- Appinventiv. (2024). Top AI Trends in 2024: Transforming Businesses Across Industries. Retrieved from [Appinventiv](https://appinventiv.com/blog/ai-trends/)

In [12]:
Markdown(output_report["output"])

# Current Trends in AI: Focus on Large Language Models (LLMs) for Education

## Introduction

Artificial Intelligence (AI) has been a transformative force across various sectors, and its impact on education is becoming increasingly significant. Among the various AI technologies, Large Language Models (LLMs) like OpenAI's GPT-4 have shown immense potential in revolutionizing educational practices. This report delves into the current trends in AI with a specific focus on the application of LLMs in education, highlighting their benefits, challenges, and future prospects.

## Overview of AI Trends in 2023

### Generative AI

Generative AI, which includes LLMs, has seen explosive growth in 2023. These models are capable of generating human-like text, creating images, and even producing music. The widespread adoption of generative AI tools like ChatGPT, DALL-E, and others has brought AI into mainstream use, making it accessible to a broader audience (Kesari, 2022; Heikkilä, 2023).

### Multimodal AI

Multimodal AI, which processes multiple types of data inputs such as text, images, and audio, is another significant trend. This technology aims to mimic human sensory processing, enhancing the capabilities of AI systems to understand and interact with the world more comprehensively (Craig, 2024).

### AI Democratization

The democratization of AI refers to making AI tools accessible to a wider audience, including non-technical users. This trend is driven by the development of user-friendly AI platforms and open-source frameworks, enabling more people to leverage AI for various applications (Ayuya, 2024).

### AI in Cybersecurity

AI-driven cybersecurity solutions are becoming more prevalent, offering enhanced threat detection and response capabilities. These systems use machine learning to predict, detect, and respond to cyber threats more efficiently than traditional methods (Ayuya, 2024).

### Ethical and Explainable AI

As AI systems become more integrated into daily life, there is a growing emphasis on ethical AI and explainability. This involves developing AI systems that are transparent, fair, and accountable, addressing concerns about bias and misuse (Srivastava, 2024).

## LLMs in Education

### Enhancing Learning Experiences

LLMs have the potential to transform educational experiences by providing personalized learning, automating administrative tasks, and offering new ways to engage students. These models can generate customized content, answer student queries, and provide feedback, making learning more interactive and tailored to individual needs (McKinsey, 2023).

### Personalized Learning

One of the most significant benefits of LLMs in education is their ability to offer personalized learning experiences. By analyzing student data, LLMs can tailor educational content to meet the specific needs and learning styles of each student. This personalized approach can help improve student engagement and outcomes (Baker Tilly, 2024).

### Automating Administrative Tasks

LLMs can also automate various administrative tasks, such as grading assignments, scheduling, and managing student records. This automation can free up educators' time, allowing them to focus more on teaching and interacting with students (Appinventiv, 2024).

### Enhancing Accessibility

LLMs can make education more accessible by providing real-time translation services, generating content in multiple languages, and offering support for students with disabilities. For example, AI-powered tools can convert text to speech, provide subtitles for videos, and offer other assistive technologies (Bocasay, 2023).

## Challenges and Ethical Considerations

### Data Privacy and Security

The use of LLMs in education raises significant concerns about data privacy and security. Educational institutions must ensure that student data is protected and that AI systems comply with relevant regulations and standards (TechTarget, 2024).

### Bias and Fairness

LLMs can inadvertently perpetuate biases present in their training data, leading to unfair or discriminatory outcomes. It is crucial to develop and implement strategies to mitigate these biases and ensure that AI systems are fair and equitable (Appinventiv, 2024).

### Transparency and Explainability

Ensuring that AI systems are transparent and explainable is essential for building trust among educators, students, and parents. This involves making the decision-making processes of AI systems understandable and providing clear explanations for their outputs (Srivastava, 2024).

### Ethical Use of AI

The ethical use of AI in education involves considering the broader implications of AI technologies and ensuring that they are used responsibly. This includes addressing issues such as the potential for AI to replace human educators and the impact of AI on student well-being (Baker Tilly, 2024).

## Future Prospects

### Integration with Other Technologies

The future of LLMs in education will likely involve their integration with other emerging technologies, such as augmented reality (AR), virtual reality (VR), and the Internet of Things (IoT). These integrations can create more immersive and interactive learning experiences, further enhancing the educational process (Appinventiv, 2024).

### Lifelong Learning and Professional Development

LLMs can also play a significant role in lifelong learning and professional development. By providing personalized learning paths and resources, AI can help individuals continuously update their skills and knowledge, keeping pace with the rapidly changing job market (McKinsey, 2023).

### Research and Innovation

Ongoing research and innovation in AI and LLMs will continue to drive advancements in educational technologies. This includes developing more sophisticated models, improving the accuracy and reliability of AI systems, and exploring new applications of AI in education (Heikkilä, 2023).

## Conclusion

The application of LLMs in education represents a significant trend in the current AI landscape. These models have the potential to revolutionize educational practices by offering personalized learning experiences, automating administrative tasks, and enhancing accessibility. However, it is essential to address the challenges and ethical considerations associated with their use to ensure that AI technologies are implemented responsibly and effectively. As research and innovation continue to advance, the future of LLMs in education looks promising, with the potential to transform learning experiences and outcomes for students worldwide.

## References

- Kesari, G. (2022). AI Trends For 2023: Industry Experts (And ChatGPT AI) Make Their Predictions. Forbes. Retrieved from [Forbes](https://www.forbes.com/sites/ganeskesari/2022/12/22/ai-trends-for-2023-industry-experts-and-chatgpt-ai-make-their-predictions/)
- Heikkilä, M. (2023). Four trends that changed AI in 2023. MIT Technology Review. Retrieved from [MIT Technology Review](https://www.technologyreview.com/2023/12/19/1085696/four-trends-that-changed-ai-in-2023/)
- McKinsey & Company. (2023). The state of AI in 2023: Generative AI’s breakout year. Retrieved from [McKinsey](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai-in-2023-generative-ais-breakout-year)
- Bocasay. (2023). Major AI Trends in 2023. Retrieved from [Bocasay](https://www.bocasay.com/major-ai-trends-2023/)
- TechTarget. (2024). 10 top AI and machine learning trends for 2024. Retrieved from [TechTarget](https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends)
- Ayuya, C. (2024). 10 Most Impactful AI Trends in 2024. eWeek. Retrieved from [eWeek](https://www.eweek.com/artificial-intelligence/ai-trends/)
- Baker Tilly. (2024). AI trends: 2023 recap and insights for what’s to come. Retrieved from [Baker Tilly](https://www.bakertilly.com/insights/ai-trends-2023-recap-and-insights-for-whats-to-come)
- Appinventiv. (2024). Top AI Trends in 2024: Transforming Businesses Across Industries. Retrieved from [Appinventiv](https://appinventiv.com/blog/ai-trends/)